In [1]:
import csv

In [2]:
'''class_labels = {'bottle': 44, 'person': 1, 'backpack': 27, 'handbag': 31, 
                'umbrella': 28, 'bowl': 51, 'cup': 47, 'clock': 85, 
                'knife': 49, 'spoon': 50, 'fork': 48, 'chair': 62, 
                'potted plant': 64, 'dining table': 67, 'keyboard': 76, 'mouse': 74, 
                'laptop': 73, 'cell phone': 77, 'scissors': 87, 'book': 84, 'monitor': 72}'''
class_labels = {'bottle': 1, 'person': 2, 'backpack': 3, 'handbag': 4, 
                'umbrella': 5, 'bowl': 6, 'cup': 7, 'clock': 8, 
                'knife': 9, 'spoon': 10, 'fork': 11, 'chair': 12, 
                'potted plant': 13, 'dining table': 14, 'keyboard': 15, 'mouse': 16, 
                'laptop': 17, 'cell phone': 18, 'scissors': 19, 'book': 20, 'monitor': 21}

In [3]:
filename = 'test.csv'

with open(filename, 'r') as f:
    mycsv = csv.reader(f)
    mycsv = list(mycsv)
    # 0:'filename', 1:'width', 2:'height', 3:'class', 4:'xmin', 5:'xmax', 6:'ymin', 7:'ymax'
print(mycsv[0])

['filename', 'width', 'height', 'class', 'xmin', 'xmax', 'ymin', 'ymax']


In [4]:
import os
from PIL import Image
import numpy as np

In [5]:
PATH_TO_TEST_IMAGES_DIR = '/home/linda/OLIV/training/data/images/cooc2017'
test_image_path = os.path.join(PATH_TO_TEST_IMAGES_DIR, mycsv[166][0])
image = Image.open(test_image_path)
print(np.array(image.getdata()).size)
print(image.size[0]*image.size[1])

921600
307200


In [ ]:
images = {} 
PATH_TO_TEST_IMAGES_DIR = '/home/linda/OLIV/training/data/images/cooc2017'


file = open("subsetCOCO_test_1_224.json", "w")
file.write("{")

# write images
# "file_name": "000000394940.jpg", "height": 640, "width": 426, "id":  289343
from IPython.display import display, clear_output
file.write("\"images\":[")
i = 0
invalid = 0
for row in mycsv:
    clear_output(wait=True)
    display(i)
    if i == 0: # skip
        i += 1
        continue
    test_image_path = os.path.join(PATH_TO_TEST_IMAGES_DIR, row[0])
    image = Image.open(test_image_path)
    if np.array(image.getdata()).size == image.size[0]*image.size[1]:
        invalid += 1
        continue
    if row[0] not in images:
        images[row[0]] = i
        img_id = i
        if i == 1:
            file.write("{\"filename\": \"" + row[0] + "\", \"height\": " + row[2] + ", \"width\": " + row[1] + 
                       ", \"id\": " + str(img_id) + "}")
        else:
            file.write(",{\"filename\": \"" + row[0] + "\", \"height\": " + row[2] + ", \"width\": " + row[1] + 
                       ", \"id\": " + str(img_id) + "}")
        i += 1
file.write("]")

# write annotations
# "image_id": 289343, "bbox": [xmin, ymin, width, height], "category_id": 18, "id": 1, "iscrowd": 0, "area": 1108.0955
file.write(",\"annotations\":[")
i = 0
for row in mycsv:
    if i == 0: # skip
        i += 1
        continue
    if row[0] in images:
        if i == 1:
            file.write("{\"image_id\": " + str(images[row[0]])
                       + ", \"bbox\": [" + row[4] + ", " + row[6] + ", " + str(float(row[5])-float(row[4])) + ", " + str(float(row[7])-float(row[6]))
                       + "], \"category_id\": " + str(class_labels[row[3]]) 
                       + ", \"id\": " + str(i) + ", \"iscrowd\": 0"
                       + ", \"area\": " + str((float(row[5])-float(row[4]))*(float(row[7])-float(row[6]))) + "}")
        else:
            file.write(",{\"image_id\": " + str(images[row[0]])
                       + ", \"bbox\": [" + row[4] + ", " + row[6] + ", " + str(float(row[5])-float(row[4])) + ", " + str(float(row[7])-float(row[6]))
                       + "], \"category_id\": " + str(class_labels[row[3]]) 
                       + ", \"id\": " + str(i) + ", \"iscrowd\": 0"
                       + ", \"area\": " + str((float(row[5])-float(row[4]))*(float(row[7])-float(row[6]))) + "}")
    i += 1
file.write("]")

# write categories
# "id": 18, "name": "dog"
file.write(", \"categories\": [")
i = 0
for key, value in class_labels.items():
    if i == 0:
        file.write("{\"id\": " + str(value) + ", \"name\": \"" + key + "\"}")
    else:
        file.write(",{\"id\": " + str(value) + ", \"name\": \"" + key + "\"}")
    i += 1
file.write("]")

file.write("}")
file.close()

print("invalid: %s" % invalid)

344

In [25]:
import pickle

fileObject = open('filenames_id.p','wb')
pickle.dump(images, fileObject)
fileObject.close()